In [1]:
import os
import json
import pandas as pd
from IPython.display import display, HTML

In [2]:
def find_best_config_str(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            # now, get string between the brackets { and }
                            configs = '{' + line.split('{')[1].split('}')[0] + '}'
                            # replace single quotes with double quotes
                            configs = configs.replace("'", '"')
                            # convert each boolean value to string
                            configs = configs.replace(': True', ': "True"')
                            configs = configs.replace(': False', ': "False"')
                            # convert to json
                            config = json.loads(configs)
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

In [3]:
def find_best_config_fp(exp_path, mode='retrieval', score_by='MAP'):
    l = []
    # find al files called log.txt, recursively
    for root, dirs, files in os.walk(exp_path):
        for file in files:
            if file.endswith('log.txt'):
                # open file, and for each line test if it contains the string "result for verification run - "
                with open(os.path.join(root, file)) as f:
                    for line in f:
                        if f'result for {mode} run' in line:
                            score = float(line.split(f'{score_by}: ')[1].split(' ')[0])
                            config = json.load(open(os.path.join(f.name, '..', '..', 'config.json')))
                            l.append((score, config))

    # sort the list by the first element, which is the score
    l.sort(key=lambda x: x[0], reverse=True)
    return l

exp_path = 'data-out/experiments/test-all/'

mode = 'verification'
score_by='F1'
configs = find_best_config_fp(exp_path, mode, score_by)

In [4]:
exp_path = 'data-out/experiments/test-dev'

# # print the top 10 results
# mode = 'retrieval'
# score_by='MAP'
# configs = find_best_config(exp_path, mode, score_by)
# for i, (f1, config) in enumerate(configs[:2]):
#     print(f'{i}. {score_by} score: {f1:.4f}, config: {config}')

# print the top 10 results
mode = 'verification'
score_by='Strict-F1'
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    print(f'{i}. {score_by} score: {score:.4f}, config: {config}')


0. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'True', 'fingerprint': 'nonorm-noscale-ignore_nei'}
1. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': './data-out/experiments/test-dev/retr-OPENAI/nopre-noname-nobio', 'preprocess': 'False', 'retriever_label': 'OPENAI', 'verifier_label': 'OPENAI', 'normalize_scores': 'False', 'scale': 'False', 'ignore_nei': 'False', 'fingerprint': 'nonorm-noscale-noignore_nei'}
2. Strict-F1 score: 0.9230, config: {'blind_run': 'False', 'split': 'dev', 'add_author_name': 'False', 'add_author_bio': 'False', 'retriever_k': 5, 'out_dir': '.

In [5]:
configs = find_best_config_str(exp_path, mode, score_by)
for i, (score, config) in enumerate(configs[:]):
    if config['retriever_label'] == 'TERRIER':
        print(f'{mode} - {i}. {score_by} score: {score:.4f}, config: {config["out_dir"]}/{config["fingerprint"]} - name:{config["add_author_name"]} bio:{config["add_author_bio"]} {config["verifier_label"]}')

verification - 6. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 7. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-noscale-noignore_nei - name:True bio:False OPENAI
verification - 8. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-ignore_nei - name:True bio:False OPENAI
verification - 9. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/nonorm-scale-noignore_nei - name:True bio:False OPENAI
verification - 10. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-ignore_nei - name:True bio:False OPENAI
verification - 11. Strict-F1 score: 0.8821, config: ./data-out/experiments/test-dev/retr-TERRIER/pre-name-nobio/norm-noscale-noignore_nei - name:True bio:False OPENAI
veri

In [6]:
# find best config for retrieval overall
mode = 'retrieval'
score_by='MAP'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)

# Create a list of dictionaries to store the data
data = []

for i, (f1, config) in enumerate(configs[:]):
    config_link = f'<a href="{config["out_dir"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        score_by: f1,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings", 
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        # 'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_retrieval = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_retrieval.to_html(escape=False)))

,Rank,MAP,Retrieval,Preprocess,Author Name,Author Bio
0,1,0.6877,PyTerrier,True,False,True
1,2,0.6743,Embeddings,False,False,False
2,3,0.6711,Embeddings,True,True,False
3,4,0.6594,PyTerrier,True,True,True
4,5,0.6572,PyTerrier,True,False,False
5,6,0.6432,Embeddings,True,True,True
6,7,0.6428,PyTerrier,False,True,False
7,8,0.6415,Embeddings,True,False,False
8,9,0.6407,PyTerrier,False,False,False
9,10,0.6400,Embeddings,False,False,True


In [7]:
print(df_retrieval.to_latex(index=False))

\begin{tabular}{rrlrrr}
\toprule
Rank & MAP & Retrieval & Preprocess & Author Name & Author Bio \\
\midrule
1 & 0.687700 & PyTerrier & True & False & True \\
2 & 0.674300 & Embeddings & False & False & False \\
3 & 0.671100 & Embeddings & True & True & False \\
4 & 0.659400 & PyTerrier & True & True & True \\
5 & 0.657200 & PyTerrier & True & False & False \\
6 & 0.643200 & Embeddings & True & True & True \\
7 & 0.642800 & PyTerrier & False & True & False \\
8 & 0.641500 & Embeddings & True & False & False \\
9 & 0.640700 & PyTerrier & False & False & False \\
10 & 0.640000 & Embeddings & False & False & True \\
11 & 0.637500 & PyTerrier & True & True & False \\
12 & 0.633600 & Embeddings & False & True & False \\
13 & 0.633400 & PyTerrier & False & True & True \\
14 & 0.628500 & PyTerrier & False & False & True \\
15 & 0.619600 & Embeddings & False & True & True \\
16 & 0.594800 & Embeddings & True & False & True \\
\bottomrule
\end{tabular}



In [11]:
# find best config for verification overall
mode = 'verification'
score_by='Strict-F1'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)

# Create a list of dictionaries to store the data
data = []

for i, (score, config) in enumerate(configs[0:]):
    config_link = f'<a href="{config["out_dir"]}/{config["fingerprint"]}/config.json" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        'Macro-F1': score,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings",
        'Verification': config['verifier_label'],
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        'Scale': config['scale'],
        'Normalize': config['normalize_scores'],
        'Ignore NEI': config['ignore_nei'],
        'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_verification = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_verification.to_html(escape=False)))

,Rank,Macro-F1,Retrieval,Verification,Preprocess,Author Name,Author Bio,Scale,Normalize,Ignore NEI,Config
0,1,0.9230,Embeddings,OPENAI,False,False,False,False,False,True,Link
1,2,0.9230,Embeddings,OPENAI,False,False,False,False,False,False,Link
2,3,0.9230,Embeddings,OPENAI,False,False,False,True,False,True,Link
3,4,0.9230,Embeddings,OPENAI,False,False,False,False,True,True,Link
4,5,0.9230,Embeddings,OPENAI,False,False,False,False,True,False,Link
5,6,0.9230,Embeddings,OPENAI,False,False,False,True,True,True,Link
6,7,0.8821,PyTerrier,OPENAI,True,False,True,False,False,True,Link
7,8,0.8821,PyTerrier,OPENAI,True,False,True,False,False,False,Link
8,9,0.8821,PyTerrier,OPENAI,True,False,True,True,False,True,Link
9,10,0.8821,PyTerrier,OPENAI,True,False,True,True,False,False,Link


In [ ]:
print(df_verification.to_latex(index=False))

\begin{tabular}{rrlrrrrrr}
\toprule
Rank & Macro-F1 & Retrieval & Preprocess & Author Name & Author Bio & Scale & Normalize & Ignore NEI \\
\midrule
1 & 0.923000 & Embeddings & False & False & False & False & False & True \\
2 & 0.923000 & Embeddings & False & False & False & False & False & False \\
3 & 0.923000 & Embeddings & False & False & False & True & False & True \\
4 & 0.923000 & Embeddings & False & False & False & False & True & True \\
5 & 0.923000 & Embeddings & False & False & False & False & True & False \\
6 & 0.923000 & Embeddings & False & False & False & True & True & True \\
7 & 0.882100 & PyTerrier & True & False & True & False & False & True \\
8 & 0.882100 & PyTerrier & True & False & True & False & False & False \\
9 & 0.882100 & PyTerrier & True & False & True & True & False & True \\
10 & 0.882100 & PyTerrier & True & False & True & True & False & False \\
11 & 0.882100 & PyTerrier & True & False & True & False & True & True \\
12 & 0.882100 & PyTerrier & True

In [ ]:
# find best config for verification overall
mode = 'retrieval'
score_by='MAP'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)
for i, (f1, config) in enumerate(configs[0:9]):
    print(f'{" " if i+1 < 10 else ""}{i+1}. {score_by} score: {f1:.4f}, pre:{config["preprocess"]}\tname:{config["add_author_bio"]}\tbio:{config["add_author_name"]}\tconfig: {config["out_dir"]}/config.json')

In [ ]:
# find best config for verification overall
mode = 'verification'
score_by='F1'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)
for i, (f1, config) in enumerate(configs[0:]):
    print(f'{" " if i+1 < 10 else ""}{i+1}. {score_by} score: {f1:.4f}\tconfig: {config["out_dir"]}/{config["fingerprint"]}')  

In [ ]:
# find best config for verification overall
mode = 'verification'
score_by='Strict-F1'
configs = find_best_config_fp(f'./data-out/experiments/test-dev', mode, score_by)

try:
    no_ext_data = []
    ext_data = []
    for item in configs:
        config = item[1]
        if config['add_author_name'] or config['add_author_bio']:
            ext_data.append(item)
        else:
            no_ext_data.append(item)
    
    no_ext_data = sorted(no_ext_data, key=lambda x: x[0], reverse=True)
    ext_data = sorted(ext_data, key=lambda x: x[0], reverse=True)

except:
    pass

# Create a list of dictionaries to store the data
data = []

for i, (score, config) in enumerate(no_ext_data[0:]):
    config_link = f'<a href="{config["out_dir"]}/{config["fingerprint"]}" target="_blank">Link</a>'
    data.append({
        'Rank': i + 1,
        'Macro-F1': score,
        'Retrieval': "PyTerrier" if config['retriever_label'] == "TERRIER" else "Embeddings",
        'Preprocess': config['preprocess'],
        'Author Name': config['add_author_bio'],
        'Author Bio': config['add_author_name'],
        'Scale': config['scale'],
        'Normalize': config['normalize_scores'],
        'Ignore NEI': config['ignore_nei'],
        # 'Config': config_link
    })

# Convert the list of dictionaries to a DataFrame
df_verification = pd.DataFrame(data)

# Display the DataFrame with clickable links
display(HTML(df_verification.to_html(escape=False)))

,Rank,Macro-F1,Retrieval,Preprocess,Author Name,Author Bio,Scale,Normalize,Ignore NEI
0,1,0.9230,Embeddings,False,False,False,False,False,True
1,2,0.9230,Embeddings,False,False,False,False,False,False
2,3,0.9230,Embeddings,False,False,False,True,False,True
3,4,0.9230,Embeddings,False,False,False,False,True,True
4,5,0.9230,Embeddings,False,False,False,False,True,False
5,6,0.9230,Embeddings,False,False,False,True,True,True
6,7,0.7455,Embeddings,False,False,False,True,True,False
7,8,0.7020,Embeddings,False,False,False,True,False,False
8,9,0.6612,Embeddings,False,False,False,False,False,True
9,10,0.6612,Embeddings,False,False,False,False,False,False


In [ ]:
for i, (f1, config) in enumerate(ext_data):
    print(f'{mode} using ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')

In [ ]:
for i, (f1, config) in enumerate(no_ext_data):
    print(f'{mode} no ext data - {i+1}. {score_by} score: {f1:.4f}, config: {config}')